Run
cd C:\Users\danii\Desktop\landeskunde-portal
.\umgebung1\Scripts\Activate.ps1

python -m src.pipelines.01_build_places
uvicorn src.web.api:app --reload


Test:

http://127.0.0.1:8000/completion?query=ahr

dann http://127.0.0.1:8000/place/120600050005


python -m src.services.search_places "Ahrensfelde"

Fahrplan Landeskundeportal (Rahmenplan, MVP → Ausbau) – inkl. KI-Einsatz + Modellvorschlägen
0) Zielbild (was am Ende “funktioniert”)

Ein Suchfeld: Nutzer tippt Ahrensfelde industrie 1800

System macht:

Place Resolution: “Ahrensfelde” → eindeutige Orts-Entität (ARS/AGS/Gemeindeverzeichnis/GOV)

Query Decomposition: Topic/Zeitraum/Facetten extrahieren (Industrie, 19. Jh/1800er)

Ergebnisse:

Ortsseite (kuratiert/erklärend): Steckbrief + Zeitstrahl + Themenblöcke

Weiterverweise: Treffer aus Institutionen/Datengebern (Museum, Denkmalamt, Archiv, Bibliothek etc.) inkl. Snippets & Links

Belegpflicht: Alles Generierte ist mit zitierfähigen Evidence-Objekten hinterlegt.

1) Daten- und Systemarchitektur (ohne relationale DB)
1.1 “Place Store” (kleiner, stabiler Kernbestand)

Zweck: einheitliche Orts-ID, Namenvarianten, Hierarchie, Geo

Inhalt:

Gemeindeverzeichnis BB (CSV/XLSX) → places.jsonl

Optional Ergänzung: GOV IDs, GeoNames/TGN IDs, Denkmal-Gazetteer IDs

Speicherform:

JSONL + SQLite/duckdb optional nur als Cache, nicht als “Master”

Suchindex: Meilisearch / OpenSearch / Elasticsearch (empfohlen für später)

KI: keine (hier maximal Regeln + Fuzzy Matching)

1.2 “Source Registry” (Metadaten über Datengeber)

Zweck: welche Quellen gibt es, welche Lizenzen, welche Endpunkte, welche Felder

Inhalt je Quelle:

Name, Betreiber, Lizenz, API/Harvest-Typ (REST/OAI/IIIF), Rate Limits

Mappings: “welches Feld entspricht Ort/Datum/Objekttyp”

Speicherform: YAML/JSON (konfigurierbar)

KI: keine

1.3 “Evidence Store” (Belege als kleinste Wahrheitseinheit)

Zweck: Zitierfähigkeit + Nachvollziehbarkeit (anti-Halluzination)

Evidence-Objekt:

source_id, source_url, title, snippet

where (page/anchor/iiif canvas/timecode)

place_candidates (mit score), time_span (falls ableitbar), topics/tags

Speicherform: JSONL (oder Dokumentenspeicher)

KI: punktuell (Extraktion/Tagging), aber Evidence bleibt “roh + verlinkt”

1.4 “Retrieval Layer”

Zweck: schnelle Suche über Evidence

Komponenten:

Volltextindex (OpenSearch/Elastic/Meili)

Optional: Vektorindex (Qdrant/FAISS)

KI: Embeddings (optional, aber sehr nützlich bei heterogenen Daten)

1.5 “Ortsseiten-Generator”

Zweck: aus Evidence + Place-Daten eine Leseseite bauen

KI: RAG/Controlled Generation (mit strikten Zitaten)

2) Ingestion / Harvest (Prozessfahrplan)
2.1 Phase A – MVP Ingestion (nur 2–3 Quellen)

Gemeindeverzeichnis → Place Store ✅

Denkmalamt Search API → Evidence (nur Trefferliste + Snippet + Link)

museum-digital → Evidence (Search + Objektmetadaten)

Optional: Ortslexikon PDF/TXT (nur für 1 Ort) → Evidence

KI in Phase A

OCR/HTR (falls PDF/Scan):

Transkribus HTR (falls historisches Druckbild gut passt)

Tesseract als Minimalvariante

Text Normalisierung/Chunking:

regelbasiert + heuristisch (Seitenmarker “=== Seite X ===”)

2.2 Phase B – Ausbau Ingestion (mehr Quellen, stabiler)

OAI-PMH (z.B. museum-digital, Bibliotheken)

IIIF (Bilder/Manuskripte)

Archiv-Findmittel (EAD/XML)

Wikidata/DBpedia optional als “Context”, nicht als Primärbeleg

Regelmäßiger Harvest (Scheduler)

KI in Phase B

Entity Linking (Orte/Personen) aus Fließtext

Zeitextraktion (Zeiträume/Datierungen)

3) KI-Einsatzpunkte (wo genau, mit Modellen)
3.1 Place Resolution (Ort korrekt erkennen)

Ziel: “Ahrensfelde” → richtige Entität + Hierarchie + Koordinate

Primär: Regeln + Gazetteer

String-Fuzzy (rapidfuzz)

Hierarchie-Regeln (Landkreis/Gemeinde/Ortsteil)

Optional KI:

NER/Entity Linking in Fließtexten (Ortslexikon, Beschreibungen)

Modelle

NER (deutsch):

spaCy de_core_news_lg (schnell, solide)

oder flair NER (German) (oft stärker, aber schwerer)

Entity Linking (optional später):

RAG-gestützt: Kandidatenliste aus Gazetteer + LLM entscheidet mit Regeln

Wichtig: keine “freie” KI-Entscheidung ohne Kandidatenliste.

3.2 Query Understanding (Topic + Zeitraum + Facetten)

Ziel: aus industrie 1800 wird {topic: industrie, time: 1800–1899}

Methode:

Regeln + kleine Klassifikatoren

Optional LLM für robuste Normalisierung

Modelle

LLM (lokal/optional cloud):

Lokal: Llama 3.1 8B Instruct (guter Baseline für Deutsch)

Alternativ: Qwen2.5 7B Instruct (stark, häufig sehr brauchbar)

Zeitparser (nicht-KI):

dateparser + eigene Regeln (19. Jh, 1800er etc.)

3.3 OCR/HTR / ASR (Erschließung unstrukturierter Medien)

OCR (Scans, PDFs):

Transkribus HTR (bei geeigneten Layouts/Schriften sehr gut)

ABBYY (falls verfügbar, stark)

Tesseract (MVP)

ASR (Audio/Video):

Whisper large-v3 (offline möglich; sehr robust)

3.4 Enrichment / Tagging (nur als Zusatz, nicht als Quelle)

Ziel: Evidence bekommt Topics/Tags, grobe Typen, Personen/Orte/Zeiten

Aufgaben:

Topic Labels (Industrie, Landwirtschaft, Krieg, Religion …)

Named Entities (Person, Ort, Institution)

Event-/Objekttypen (z.B. “Fabrik”, “Dorf”, “Kirche”)

Modelle

Embeddings:

multilingual: bge-m3 (stark, universell)

deutsch: intfloat/multilingual-e5-large (sehr verbreitet)

Klassifikation/Labeling:

LLM (Llama/Qwen) mit constrained output (JSON Schema)

oder kleiner Textklassifikator (sklearn) sobald Trainingsdaten existieren

3.5 Retrieval Augmented Generation (Ortsseite schreiben)

Ziel: aus Evidence + Place-Daten eine lesbare Ortsseite erzeugen

Striktes Prinzip:

Nur schreiben, was belegt ist

Jede Aussage bekommt Evidence-IDs

“Unklar/unsicher” wird ausdrücklich markiert

Modelle (Generation)

Lokal:

Llama 3.1 8B Instruct / Qwen2.5 7B Instruct

Wenn später höhere Qualität nötig:

Größere Instruct-Modelle (14B/32B) lokal, falls Hardware ok

Prompting/Guardrails:

JSON-Schema Output + citation fields

“If no evidence, say ‘keine Belege’ ”

4) MVP Scope (konkret, in 2–4 Wochen realistisch)
4.1 MVP-Funktionen

Suche: place + topic/time (ein Feld)

Place Resolution über Place Store (Gemeindeverzeichnis)

Ortsseite:

Steckbrief (aus Place Store)

“Historisches Ortslexikon” Abschnitt (nur für 1–2 Orte als Demo)

Weiterverweise:

Denkmalamt Search API (Trefferliste)

museum-digital Search API (Trefferliste)

Evidence:

Treffer speichern als Evidence JSON (inkl. URL, snippet, source_id)

4.2 MVP-Qualitätsregeln

Keine generierten Behauptungen ohne Evidence

Zeitangaben nur, wenn aus Quelle ableitbar

Matching-Confidence anzeigen (z.B. “Ortssicherheit: hoch/mittel/niedrig”)

5) Ausbauplan (nach MVP)
5.1 Phase 2 – Mehr Orte, mehr Quellen, besseres Matching

GOV Integration (REST/SOAP) als zusätzliche Orts-IDs & Namenshistorie

Gazetteer vom Denkmalamt als zweiter Ortskatalog → Crosswalk:

Place Store: ars/ags + koordinaten + name_variants

Mapping: nearest neighbor + name similarity + Landkreis-Filters

Mehr Harvest:

OAI-PMH (Sammlungen)

IIIF (Digitalisate)

Index:

Volltext + Vektorindex

5.2 Phase 3 – Redaktion & Kuratierung

Redaktionsworkflow:

“Evidence prüfen”

“Ortsseite freigeben”

Feedbackkanal:

Nutzer meldet falsche Zuordnung / fehlende Quelle

6) Minimaler Komponenten-Stack (nur zur Orientierung, kein Code)

Backend: Python (FastAPI)

Place Store: JSONL (+ optional sqlite/duckdb cache)

Index: Meilisearch (MVP) → später OpenSearch

Vector Store (optional): Qdrant

KI:

OCR: Transkribus/Tesseract

ASR: Whisper large-v3

Embeddings: bge-m3 oder multilingual-e5-large

LLM: Llama 3.1 8B Instruct oder Qwen2.5 7B Instruct